In [3]:
%load_ext autoreload
%autoreload 2


import modish
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline


folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities_motifs/bayesian'
!mkdir $folder
import flotilla
flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects'

study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian', flotilla_dir=flotilla_dir)
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=20))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities_motifs/bayesian': No such file or directory
2015-12-04 10:14:21	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities_bayesian/datapackage.json
2015-12-04 10:14:21	Parsing datapackage to create a Study object
2015-12-04 10:15:06	Initializing Study
2015-12-04 10:15:06	Initializing Predictor configuration manager for Study
2015-12-04 10:15:06	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2015-12-04 10:15:06	Added ExtraTreesClassifier to default predictors
2015-12-04 10:15:06	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2015-12-04 10:15:06	Added ExtraTreesRegressor to default predictors
2015-12-04 10:15:06	Predictor GradientBoostingClassifier is of typ

In [4]:
study.supplemental.modalities_tidy.groupby(['phenotype', 'modality']).size()

phenotype  modality  
MN         bimodal        390
           multimodal       1
           ~0             566
           ~1            1639
NPC        bimodal        799
           multimodal       2
           ~0            1092
           ~1            3047
iPSC       bimodal       1383
           middle           1
           multimodal      33
           ~0            1605
           ~1            4611
dtype: int64

### Make BED files of each modality within each celltype

Use the other modalities, within that celltype, as background.

  - What properties are unique to events that are bimodal in iPSC, rather than ~0 or ~1?
    - Foreground: Bimodal events in iPSC
    - Background: All other events in iPSC

In [5]:
# %%time 

# bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities'
# ! mkdir $bed_folder
# import pybedtools

# DIR = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

# exon_bedfile = '{}/exon2.bed'.format(DIR)
# exon_bed = pybedtools.BedTool(exon_bedfile)



# for phenotype, phenotype_df in study.supplemental.modalities_tidy.groupby('phenotype'):
#     background_events = set(phenotype_df.event_name)
#     for modality, modality_df in phenotype_df.groupby('modality'):
#         event_names = set(modality_df.event_name)
#         format_args = bed_folder, phenotype, modality
#         filtered_bed = exon_bed.filter(lambda x: x.name in event_names)
#         filtered_filename = '{}/exon2_{}_{}.bed'.format(*format_args)
#         filtered_bed.saveas(filtered_filename)

#         background_bed = exon_bed.filter(lambda x: x.name not in event_names and x.name in background_events)
#         background_filename = '{}/exon2_{}_{}_background.bed'.format(*format_args)
#         background_bed.saveas(background_filename)

#         ! wc -l $filtered_filename
#         ! wc -l $background_filename

In [6]:


bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian'
! mkdir $bed_folder
import pybedtools

mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian': File exists


In [7]:
import pyhomer

In [8]:
import os

homer_dir = '{}/homer'.format(bed_folder)
!mkdir -p $homer_dir

DIRECTIONS = 'upstream', 'downstream'


import pybedtools

DIR = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

exon_bedfile = '{}/exon2.bed'.format(DIR)
exon_bed = pybedtools.BedTool(exon_bedfile)

commands = []

findMotifsGenome = '/home/yeo-lab/software/homer/bin/findMotifsGenome.pl'
n_processors = 4
homer_flags = '-rna -len 4,5,6 -mset vertebrates -mis 0 -p {} -noweight'.format(n_processors)


primate_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPrimates.bed'
primate = pybedtools.BedTool(primate_filename)
placental_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed'
placental = pybedtools.BedTool(placental_filename)
conserved_regions = {'primate': primate, 'placental': placental}

genome = 'hg19'

nt = 400


for phenotype, phenotype_df in study.supplemental.modalities_tidy.groupby('phenotype'):
    background_events = set(phenotype_df.event_name)
    for modality, modality_df in phenotype_df.groupby('modality'):
        event_names = set(modality_df.event_name)
        format_args = bed_folder, phenotype, modality
        foreground_bed = exon_bed.filter(lambda x: x.name in event_names)
        foreground_filename = '{}/exon2_{}_{}_foreground.bed'.format(*format_args)
        foreground = pybedtools.BedTool(foreground_bed.saveas(foreground_filename))

        background_bed = exon_bed.filter(lambda x: x.name not in event_names and x.name in background_events)
        background_filename = '{}/exon2_{}_{}_background.bed'.format(*format_args)
        background = pybedtools.BedTool(background_bed.saveas(background_filename))
        
        pair = pyhomer.ForegroundBackgroundPair(foreground, background)
        print '\n', pair 
        
        suffix = os.path.basename(pair.foreground.fn).replace('_foreground.bed', '')
        out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)
#         command = pair.homer(homer_flags, findMotifsGenome, out_dir=out_dir)
#         commands.append(command)

        beds = {'background': background_bed, 'foreground': foreground_bed}
        
        for clade, conservation_bed in conserved_regions.items():
            conserved_pair = pair.intersect(conservation_bed, clade)
            print '\n', conserved_pair
            suffix = os.path.basename(conserved_pair.foreground.fn).replace('_foreground.bed', '')
            out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)
            command = pair.homer(homer_flags, findMotifsGenome, out_dir=out_dir)
            commands.append(command)
        
        for direction in DIRECTIONS:
            intron_pair = pair.flanking_intron('downstream', 'hg19', 400)
            print '\n', intron_pair
            suffix = os.path.basename(intron_pair.foreground.fn).replace('_foreground.bed', '')
            out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)
#             command = pair.homer(homer_flags, findMotifsGenome, out_dir=out_dir)
#             commands.append(command)

            for clade, conservation_bed in conserved_regions.items():
                conserved_introns = intron_pair.intersect(conservation_bed, clade)
                print '\n', conserved_introns
                suffix = os.path.basename(conserved_introns.foreground.fn).replace('_foreground.bed', '')
                out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)
                command = pair.homer(homer_flags, findMotifsGenome, out_dir=out_dir)
                commands.append(command)


ForegroundBackgroundPair:
Foreground: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian/exon2_MN_bimodal_foreground.bed (390) entries
Background: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian/exon2_MN_bimodal_background.bed (2206 entries)

ForegroundBackgroundPair:
Foreground: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian/exon2_MN_bimodal_placental_foreground.bed (599) entries
Background: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian/exon2_MN_bimodal_placental_background.bed (3763 entries)

ForegroundBackgroundPair:
Foreground: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian/exon2_MN_bimodal_primate_foreground.bed (297) entries
Background: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian/exon2_MN_bimodal_primate_background.bed (1650 entries)

ForegroundBackgroundPair:
Foreground: /projects/ps-yeolab/obotvinnik/singlecell_p

In [9]:
len(commands)

78

In [13]:
ls $homer_dir

exon2_NPC_~0_downstream400_placental/  exon2_iPSC_multimodal_primate/
exon2_iPSC_multimodal_placental/


In [11]:
from gscripts.qtools import Submitter

jobname = 'homer'
sub = Submitter(commands, jobname, walltime='0:30:00', queue='home-scrm',
               ppn=n_processors, write_and_submit=True, array=True,
               out_filename='{}/{}.out'.format(homer_dir, jobname),
               err_filename='{}/{}.err'.format(homer_dir, jobname),)

running 78 tasks as an array-job.
job ID: 3839211


In [12]:
len(commands)

78